In [ ]:
from parse import read_results
from compute import *
import os

data = read_results(os.path.join('/','app', 'data'))
data = compute_metrics(data)
data = columns_to_rows(data)
data

In [ ]:

df = data.reset_index()
display(df[(df['project'] == 'jasml') & (df['framework'] == 'WALA') & (df['scope'] == 'edge') & (df['package'] == 'package') & (df['metric'] == 'Static CG Size')])
display(df[(df['project'] == 'jasml') & (df['framework'] == 'WALA') & (df['scope'] == 'edge') & (df['package'] == 'package') & (df['metric'] == 'Recall')])


In [ ]:
import seaborn as sns
from matplotlib import pyplot, ticker, rcParams
import math

sns.set_theme(style="whitegrid", palette='colorblind')
sns.color_palette("colorblind")

def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.1f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])
rcParams["axes.labelsize"] = 18
df = data.reindex(pd.MultiIndex.from_product(data.index.levels)).reset_index()
# df = data.reset_index()
nan_num = 0.00001
for scope in ['method']:
    for package in ['package']:
        grid = sns.FacetGrid(df[(df["scope"] == scope) & (df["package"] == package) & (df["project"] == "axion") & (df["framework"] == "WALA")],
                             #row='project',
                             #row_order=['axion', 'batik', 'jasml', 'xerces'],
                             col='metric',
                             col_order=['Precision', 'Recall', 'Static CG Size'],
                             sharex="col",
                             margin_titles=True,
                             legend_out=False,
                             height=1.4,
                             aspect=1.6,
                             gridspec_kws={"wspace":0.5, "hspace":0.0})
        grid.map_dataframe(sns.barplot,
                           y='algo',
                           x='metric_value',
                           hue='framework',
                           #hue_order=['OPAL', 'WALA', 'Soot', 'Doop'],
                           order=['CHA', 'RTA', '0-CFA'],
                           palette='colorblind',
                           width=0.6,
                           dodge=True)
        # grid.despine()
        for axis in grid.axes:
            axis[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            axis[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            axis[2].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '0' if x == 0 else '{:d}K'.format(int(x/1000))))
            for ax in axis:
                ax.grid(which='major', color='#ABAEAC', linewidth=1)
                ax.minorticks_on()
                for container in ax.containers:
                    for bar in container:
                        ax.annotate(human_format(bar.get_width()), size=14, xy=(bar.get_width(),bar.xy[1]), xytext=(1.3, -12.5), textcoords="offset points")

        grid.set_yticklabels(["  CHA", "  RTA", "0-CFA"], rotation=0, verticalalignment="center", fontsize=18)
        grid.despine(left=True, bottom=True)
        grid.set_axis_labels(scope.capitalize() + '/' + package.capitalize(), "Algorithm")
        grid.set_ylabels(label='')
        grid.set(xlabel=None)
        if scope == 'method':
            grid.set_titles(col_template="{col_name}", row_template="")
        if scope == 'edge':
            grid.set_titles(col_template="{col_name}", row_template="{row_name}")
            grid.set_yticklabels([])
            if package == 'package':
                grid.add_legend()
        #grid.fig.tight_layout()
        #pyplot.tight_layout()
        #grid.add_legend(ncols=2)
        #sns.move_legend(grid, "lower right", bbox_to_anchor=(0.92,0.127), ncols=1)
        pyplot.savefig('/out/axion-wala-method-package.pdf', dpi=300, bbox_inches="tight")
        pyplot.show()

In [ ]:
import seaborn as sns
from matplotlib import pyplot, ticker, rcParams
import math

def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.1f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

sns.set_theme(style="whitegrid", palette='colorblind')
sns.color_palette("colorblind")
rcParams["axes.labelsize"] = 18
df = data.reindex(pd.MultiIndex.from_product(data.index.levels)).reset_index()
# df = data.reset_index()
nan_num = 0.00001
for scope in ['edge']:
    for package in ['all']:
        grid = sns.FacetGrid(df[(df["scope"] == scope) & (df["package"] == package) & (df["algo"] == "CHA") & (df["framework"] != "Soot") & (df["framework"] != "Doop")],
                             row='project',
                             row_order=['axion', 'batik', 'jasml', 'xerces'],
                             col='metric',
                             col_order=['Precision', 'Recall', 'Static CG Size'],
                             sharex="col",
                             margin_titles=True,
                             legend_out=False,
                             height=1,
                             aspect=2.6,
                             gridspec_kws={"wspace":0.25, "hspace":0.0})
        grid.map_dataframe(sns.barplot,
                           y='algo',
                           x='metric_value',
                           hue='framework',
                           hue_order=['OPAL', 'WALA'],
                           #order=['CHA', 'RTA', '0-CFA'],
                           palette='colorblind',
                           width=0.6)
        # grid.despine()
        for axis in grid.axes:
            axis[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: "0%" if(x == 0) else str(round(x, 1)) + "%"))
            axis[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            axis[2].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '0' if x == 0 else '{:d}M'.format(int(x/1000000))))

            bar_cnt = 1
            special = [5,6,11,12,17,18,23,24]
            for ax in axis:
                ax.grid(which='major', color='#ABAEAC', linewidth=1)
                ax.minorticks_on()
                
                for container in ax.containers:
                    
                    for bar in container:
                        if bar_cnt in special:
                            ax.annotate(human_format(bar.get_width()), size=14, xy=(bar.get_width(),bar.xy[1]), xytext=(-33, -12.5), color="white", textcoords="offset points")
                        else:
                            ax.annotate(human_format(bar.get_width()), size=14, xy=(bar.get_width(),bar.xy[1]), xytext=(1.3, -12.5), textcoords="offset points")
                        bar_cnt = bar_cnt + 1

        grid.despine(left=True, bottom=True)
        grid.set(xlabel=None)
        grid.set_ylabels(label='')
        if scope == 'method':
            grid.set_titles(col_template="{col_name}", row_template="")
        if scope == 'edge':
            grid.set_titles(col_template="{col_name}", row_template="{row_name}")
            grid.set_yticklabels([])
            if package == 'package':
                grid.add_legend()
        grid.fig.tight_layout()
        pyplot.tight_layout()
        grid.add_legend(ncols=3)
        sns.move_legend(grid, "lower left", bbox_to_anchor=(0.23,0.32), ncols=1)
        pyplot.savefig('/out/cha-edge-all.pdf', dpi=300)
        pyplot.show()

In [ ]:
import seaborn as sns
from matplotlib import pyplot, ticker, rcParams
import math

def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.1f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

sns.set_theme(style="whitegrid", palette='colorblind')
sns.color_palette("colorblind")
rcParams["axes.labelsize"] = 18
df = data.reindex(pd.MultiIndex.from_product(data.index.levels)).reset_index()
# df = data.reset_index()
nan_num = 0.00001
for scope in ['edge']:
    for package in ['all']:
        grid = sns.FacetGrid(df[(df["scope"] == scope) & (df["package"] == package) & (df["algo"] == "0-CFA")],
                             row='project',
                             row_order=['axion', 'batik', 'jasml', 'xerces'],
                             col='metric',
                             col_order=['Precision', 'Recall', 'Static CG Size'],
                             sharex="col",
                             margin_titles=True,
                             legend_out=False,
                             height=1.5,
                             aspect=1.7,
                             gridspec_kws={"wspace":0.25, "hspace":0.0})
        grid.map_dataframe(sns.barplot,
                           y='algo',
                           x='metric_value',
                           hue='framework',
                           hue_order=['OPAL', 'WALA', 'Doop'],
                           #order=['CHA', 'RTA', '0-CFA'],
                           palette='colorblind',
                           width=0.6)
        # grid.despine()
        for axis in grid.axes:
            axis[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            axis[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            axis[2].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '0' if x == 0 else '{:d}K'.format(int(x/1000))))

            for ax in axis:
                ax.grid(which='major', color='#ABAEAC', linewidth=1)
                ax.minorticks_on()
                for container in ax.containers:
                    for bar in container:
                        trunc = (human_format(bar.get_width()) == "605.3K")
                        ax.annotate(human_format(bar.get_width()), size=14, color = "white" if trunc else "black", xy=(bar.get_width(),bar.xy[1]), xytext=(-46 if trunc else 1.3, -12.5), textcoords="offset points")

        grid.despine(left=True, bottom=True)
        grid.set(xlabel=None)
        grid.set_ylabels(label='')
        if scope == 'method':
            grid.set_titles(col_template="{col_name}", row_template="")
        if scope == 'edge':
            grid.set_titles(col_template="{col_name}", row_template="{row_name}")
            grid.set_yticklabels([])
            if package == 'package':
                grid.add_legend()
        grid.fig.tight_layout()
        pyplot.tight_layout()
        grid.add_legend(ncols=3)
        sns.move_legend(grid, "lower right", bbox_to_anchor=(0.91,0.425), ncols=1)
        pyplot.savefig('/out/0cfa-only.pdf', dpi=300)
        pyplot.show()

In [ ]:
import seaborn as sns
from matplotlib import pyplot, ticker, rcParams
import math

sns.set_theme(style="whitegrid", palette='colorblind')
sns.color_palette("colorblind")

def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.1f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])
rcParams["axes.labelsize"] = 18
df = data.reindex(pd.MultiIndex.from_product(data.index.levels)).reset_index()
# df = data.reset_index()
nan_num = 0.00001
for scope in ['method']:
    for package in ['package']:
        grid = sns.FacetGrid(df[(df["scope"] == scope) & (df["package"] == package) & (df["project"] == "xerces") & (df["metric"] != "Precision")],
                             #row='project',
                             #row_order=['axion', 'batik', 'jasml', 'xerces'],
                             col='metric',
                             col_order=['Recall', 'Static CG Size'],
                             sharex="col",
                             margin_titles=True,
                             legend_out=False,
                             height=5,
                             aspect=0.7,
                             gridspec_kws={"wspace":0.5, "hspace":0.0})
        grid.map_dataframe(sns.barplot,
                           y='algo',
                           x='metric_value',
                           hue='framework',
                           hue_order=['OPAL', 'WALA', 'Soot', 'Doop'],
                           order=['CHA', 'RTA', '0-CFA'],
                           palette='colorblind',
                           width=0.6,
                           dodge=True)
        # grid.despine()
        for axis in grid.axes:
            axis[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            #axis[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            axis[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '0' if x == 0 else '{:d}K'.format(int(x/1000))))
            for ax in axis:
                ax.grid(which='major', color='#ABAEAC', linewidth=1)
                ax.minorticks_on()
                for container in ax.containers:
                    for bar in container:
                        ax.annotate(human_format(bar.get_width()), size=14, xy=(bar.get_width(),bar.xy[1]), xytext=(1.3, -12.5), textcoords="offset points")

        grid.set_yticklabels(["  CHA", "  RTA", "0-CFA"], rotation=90, verticalalignment="center", fontsize=18)
        grid.despine(left=True, bottom=True)
        grid.set_axis_labels(scope.capitalize() + '/' + package.capitalize(), "Algorithm")
        grid.set_ylabels(label='')
        grid.set(xlabel=None)
        if scope == 'method':
            grid.set_titles(col_template="{col_name}", row_template="")
        if scope == 'edge':
            grid.set_titles(col_template="{col_name}", row_template="{row_name}")
            grid.set_yticklabels([])
            if package == 'package':
                grid.add_legend()
        #grid.fig.tight_layout()
        #pyplot.tight_layout()
        grid.add_legend(ncols=2)
        sns.move_legend(grid, "lower right", bbox_to_anchor=(0.92,0.127), ncols=1)
        pyplot.savefig('/out/xerces-filtered.pdf', dpi=300, bbox_inches="tight")
        pyplot.show()

In [ ]:
import seaborn as sns
from matplotlib import pyplot, ticker
import math

sns.set_theme(style="whitegrid", palette='colorblind')
sns.color_palette("colorblind")

def human_format(num):
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    # add more suffixes if you need them
    return '%.1f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

df = data.reindex(pd.MultiIndex.from_product(data.index.levels)).reset_index()
# df = data.reset_index()
nan_num = 0.00001
for scope in ['method', 'edge']:
    for package in ['all', 'package']:
        grid = sns.FacetGrid(df[(df["scope"] == scope) & (df["package"] == package)],
                             row='project',
                             row_order=['axion', 'batik', 'jasml', 'xerces'],
                             col='metric',
                             col_order=['Precision', 'Recall', 'Static CG Size'],
                             sharex="col",
                             margin_titles=True,
                             legend_out=False,
                             height=1.5,
                             aspect=1.7)
        grid.map_dataframe(sns.barplot,
                           y='algo',
                           x='metric_value',
                           hue='framework',
                           hue_order=['OPAL', 'WALA', 'Soot', 'Doop'],
                           order=['CHA', 'RTA', '0-CFA'],
                           palette='colorblind')
        # grid.despine()
        for axis in grid.axes:
            axis[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            axis[1].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:d}%'.format(int(x))))
            axis[2].xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '0' if x == 0 else '{:d}K'.format(int(x/1000))))
            for ax in axis:
                ax.grid(which='major', color='#DDDDDD', linewidth=0.8)
                ax.minorticks_on()
                for container in ax.containers:
                    for bar in container:
                        ax.annotate(human_format(bar.get_width()), size=6, xy=(bar.get_width(),bar.xy[1]), xytext=(1, -4.5), textcoords="offset points")
                    #labels = ['' if v == nan_num else human_format(v) for v in container.datavalues]
                    #ax.bar_label(container,labels=labels,size=7)

        grid.despine(left=True, bottom=True)
        grid.set_axis_labels(scope.capitalize() + '/' + package.capitalize(), "Algorithm")
        grid.set_ylabels(label='')
        if scope == 'method':
            grid.set_titles(col_template="{col_name}", row_template="")
        if scope == 'edge':
            grid.set_titles(col_template="{col_name}", row_template="{row_name}")
            grid.set_yticklabels([])
            if package == 'package':
                grid.add_legend()
                sns.move_legend(grid, "lower right", bbox_to_anchor=(0.92,0.325))
        grid.fig.tight_layout()
        pyplot.tight_layout()
        pyplot.savefig("/out/" + scope+'-'+package+'.pdf', dpi=300)
        pyplot.show()